In [ ]:
!pip install evaluate
!pip install --upgrade transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.0 MB/s eta 0:00:00


In [ ]:
#导入库
from datasets import load_dataset,DatasetDict
from transformers import AutoTokenizer,AutoModelForSequenceClassification
from transformers import TrainingArguments,Trainer
from peft import LoraConfig,get_peft_model
import evaluate
import numpy as np
import torch

In [ ]:
#全局变量
MODEL_NAME="google-bert/bert-base-uncased"
OUTPUT_DIR="./imdbBERT"
SEED=42


In [ ]:
#导入数据
row_dataset=load_dataset("imdb")
print(row_dataset)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [ ]:
#分析处理数据
split=row_dataset["train"].train_test_split(test_size=0.1,seed=SEED)
dataset=DatasetDict({
    "train":split["train"],
    "valid":split["test"],
    "test":row_dataset["test"],
})
#隐含bug，内部构造的
words_count=[len(s.split())for s in dataset["train"]["text"]]
print(np.mean(words_count))
print(np.max(words_count))
print(np.percentile(words_count,90))

234.39706666666666
2470
459.0


In [ ]:
# 对数据集进行分词
tokenizer=AutoTokenizer.from_pretrained(MODEL_NAME)
def tokenize_function(examples):
    return tokenizer(examples["text"],padding="max_length",truncation=True,max_length=256)

tokenized=dataset.map(tokenize_function,batched=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/22500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [ ]:
#lora配置
lora_config=LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query","key","value"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS",
)

In [ ]:
#加载基础模型
base_model=AutoModelForSequenceClassification.from_pretrained(MODEL_NAME,num_labels=2)

# 应用lora配置
model=get_peft_model(base_model,lora_config)
model.print_trainable_parameters()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 443,906 || all params: 109,927,684 || trainable%: 0.4038


In [ ]:
#计算指标
accuracy=evaluate.load("accuracy")
def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds=np.argmax(preds,axis=1)
    return accuracy.compute(predictions=preds, references=labels)

In [ ]:
training_args=TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=96,
    per_device_eval_batch_size=96,
    num_train_epochs=3,
    logging_steps=100,
    save_steps=100,
    eval_strategy="steps",
    eval_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    save_total_limit=1,
    seed=SEED,
    report_to="none", # 不上报到wandb等
    )
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["valid"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    )
trainer.train()

/tmp/ipython-input-3718567746.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer=Trainer(


Step,Training Loss,Validation Loss,Accuracy
100,0.648600,0.495348,0.774400
200,0.379700,0.304967,0.874800
300,0.318400,0.285390,0.881200
400,0.298400,0.273702,0.882800
500,0.291400,0.273253,0.883600
600,0.286600,0.269710,0.884400
700,0.276200,0.268685,0.884800


TrainOutput(global_step=705, training_loss=0.35615120779537984, metrics={'train_runtime': 2373.8045, 'train_samples_per_second': 28.435, 'train_steps_per_second': 0.297, 'total_flos': 8926022292480000.0, 'train_loss': 0.35615120779537984, 'epoch': 3.0})

In [ ]:
eval_res=trainer.evaluate(tokenized["test"])
print("test eval res:",eval_res)

test eval res: {'eval_loss': 0.2618216276168823, 'eval_accuracy': 0.8906, 'eval_runtime': 350.5905, 'eval_samples_per_second': 71.308, 'eval_steps_per_second': 0.744, 'epoch': 3.0}


In [ ]:
from google.colab import drive
drive.mount('/content/drive')  # 按提示完成授权

Mounted at /content/drive


In [ ]:
!cp -r /content/imdbBERT /content/drive/MyDrive/imdbBERT